In [1]:
from src.system.certificate.RASM import ReachAvoidSuperMartingaleCertificate
from src.system.polyhorn_helper import CommunicationBridge
from system.toolIO import Parser

# parser = Parser(
#     "./kaushik_sample/actions.json",
#     "./kaushik_sample/states.json",
#     "./kaushik_sample/dynamics.json",
#     "./kaushik_sample/noise.json",
#     "./kaushik_sample/config.json",
# )
# parser = Parser(
#     "./kaushik_sample_linear/actions.json",
#     "./kaushik_sample_linear/states.json",
#     "./kaushik_sample_linear/dynamics.json",
#     "./kaushik_sample_linear/noise.json",
#     "./kaushik_sample_linear/config.json",
# )
parser = Parser(
    "./djordje_sample/actions.json",
    "./djordje_sample/states.json",
    "./djordje_sample/dynamics.json",
    "./djordje_sample/noise.json",
    "./djordje_sample/config.json",
)
# parser = Parser(
#     "./djordje_sample_2/actions.json",
#     "./djordje_sample_2/states.json",
#     "./djordje_sample_2/dynamics.json",
#     "./djordje_sample_2/noise.json",
#     "./djordje_sample_2/config.json",
# )
tool_input = parser.parse()
print(tool_input)

ToolInput(state_space=Space(dimension=2, inequalities='S1**2+S2**2<9', listed_space_inequalities=None), action_policy=SystemControlPolicy(action_space=Space(dimension=2, inequalities='0<=A1<=1and0<=A2<=1', listed_space_inequalities=None), state_dimension=2, maximal_degree=10, transitions=[Equation(monomials=[]), Equation(monomials=[])], verification_mode=True, generated_constants=set(), constant_founded=False), disturbance=SystemStochasticNoise(dimension=1, distribution_name='normal', distribution_generator_parameters={'mean': [0.5], 'std_dev': [0.1], 'seed': 42}, noise_generators=NormalNoiseGenerator(mean=[0.5], std_dev=[0.1], dimension=1, seed=42)), dynamics=SystemDynamics(state_dimension=2, action_dimension=2, disturbance_dimension=1, system_transformations=[Equation(monomials=[Monomial(coefficient=0.00500000000000000, variable_generators=('S1', 'S2'), power=(1, 1)), Monomial(coefficient=0.995000000000000, variable_generators=('S1',), power=(1,)), Monomial(coefficient=-0.00500000000

In [2]:
RASM = ReachAvoidSuperMartingaleCertificate(
    maximal_degree=tool_input.synthesis_config.maximal_polynomial_degree,
    state_dimension=tool_input.state_space.dimension,
)

In [3]:
RASM.define_constraints(tool_input)

In [4]:
print(RASM)

RASM Certificate: 
+	V(x) = (V1) + (S2 * V2) + (S2**2 * V3) + (S2**3 * V4) + (S2**4 * V5) + (S2**5 * V6) + (S2**6 * V7) + (S2**7
 	* V8) + (S2**8 * V9) + (S2**9 * V10) + (S2**10 * V11) + (S1 * V12) + (S1 * S2 * V13) + (S1 * S2**2
 	* V14) + (S1 * S2**3 * V15) + (S1 * S2**4 * V16) + (S1 * S2**5 * V17) + (S1 * S2**6 * V18) + (S1 *
 	S2**7 * V19) + (S1 * S2**8 * V20) + (S1 * S2**9 * V21) + (S1**2 * V22) + (S1**2 * S2 * V23) +
 	(S1**2 * S2**2 * V24) + (S1**2 * S2**3 * V25) + (S1**2 * S2**4 * V26) + (S1**2 * S2**5 * V27) +
 	(S1**2 * S2**6 * V28) + (S1**2 * S2**7 * V29) + (S1**2 * S2**8 * V30) + (S1**3 * V31) + (S1**3 *
 	S2 * V32) + (S1**3 * S2**2 * V33) + (S1**3 * S2**3 * V34) + (S1**3 * S2**4 * V35) + (S1**3 * S2**5
 	* V36) + (S1**3 * S2**6 * V37) + (S1**3 * S2**7 * V38) + (S1**4 * V39) + (S1**4 * S2 * V40) +
 	(S1**4 * S2**2 * V41) + (S1**4 * S2**3 * V42) + (S1**4 * S2**4 * V43) + (S1**4 * S2**5 * V44) +
 	(S1**4 * S2**6 * V45) + (S1**5 * V46) + (S1**5 * S2 * V47) + (S1**5 * S2**2 * V

In [5]:
for i, constraint in enumerate(RASM.get_constraints(), start=1):
    print(f"+ Constraint {i}:")
    print(" "*4, constraint.to_polyhorn_preorder())
    print()

+ Constraint 1:
     (assert (forall ((S1 Real) (S2 Real)) (=> (>= (+ (+ 9 (* -1 (* S1 S1))) (* -1 (* S2 S2))) 0) (>= (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (+ (* 1 V1) (* (* 1 S2) V2)) (* (* 1 (* S2 S2)) V3)) (* (* 1 (* S2 (* S2 S2))) V4)) (* (* 1 (* (* S2 S2) (* S2 S2))) V5)) (* (* 1 (* (* S2 S2) (* S2 (* S2 S2)))) V6)) (* (* 1 (* (* S2 (* S2 S2)) (* S2 (* S2 S2)))) V7)) (* (* 1 (* (* S2 (* S2 S2)) (* (* S2 S2) (* S2 S2)))) V8)) (* (* 1 (* (* (* S2 S2) (* S2 S2)) (* (* S2 S2) (* S2 S2)))) V9)) (* (* 1 (* (* (* S2 S2) (* S2 S2)) (* (* S2 S2) (* S2 (* S2 S2))))) V10)) (* (* 1 (* (* (* S2 S2) (* S2 (* S2 S2))) (* (* S2 S2) (* S2 (* S2 S2))))) V11)) (* (* 1 S1) V12)) (* (* (* 1 S1) S2) V13)) (* (* (* 1 S1) (* S2 S2)) V14)) (* (* (* 1 S1) (* S2 (* S2 S2))) V15)) (* (* (* 1 S1) (* (* S2 S2) (* S2 S2))) V16)) (* (* (* 1 S1) (* (* S2 S2) (* 

# Feed to Polyhorn

In [6]:
input_string = CommunicationBridge.get_input_string(
    tool_io=tool_input,
    certificate=RASM,
)
input_config = CommunicationBridge.get_input_config(
    tool_io=tool_input,
)

In [7]:
print(input_string)

(declare-const V38 Real)
(declare-const V3 Real)
(declare-const V63 Real)
(declare-const V20 Real)
(declare-const V32 Real)
(declare-const V59 Real)
(declare-const V24 Real)
(declare-const V45 Real)
(declare-const V29 Real)
(declare-const V7 Real)
(declare-const V58 Real)
(declare-const V56 Real)
(declare-const V61 Real)
(declare-const V43 Real)
(declare-const V27 Real)
(declare-const V4 Real)
(declare-const V55 Real)
(declare-const V12 Real)
(declare-const V52 Real)
(declare-const V54 Real)
(declare-const V22 Real)
(declare-const V35 Real)
(declare-const V30 Real)
(declare-const V10 Real)
(declare-const V18 Real)
(declare-const V25 Real)
(declare-const V8 Real)
(declare-const V33 Real)
(declare-const V34 Real)
(declare-const V17 Real)
(declare-const V19 Real)
(declare-const V23 Real)
(declare-const V36 Real)
(declare-const V39 Real)
(declare-const V60 Real)
(declare-const V1 Real)
(declare-const V65 Real)
(declare-const V64 Real)
(declare-const V49 Real)
(declare-const V41 Real)
(decl

In [8]:
CommunicationBridge.feed_to_polyhorn(config=input_config, input_string=input_string)

Running solver...
Is SAT: unsat
Model: {}
